In [1]:
import tensorflow as tf
import keras.backend as K
from kerassurgeon import identify
from kerassurgeon.operations import delete_channels

import os
import cv2
import random
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageFilter

from model import mobilenet

random.seed(2021)

os.environ["CUDA_VISIBLE_DEVICES"]="4"

2022-01-18 14:25:00.502686: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow backend.


In [2]:
def get_list_from_filenames(file_path):
    with open(file_path,'r',) as f:
        lines = [one_line.strip('\n') for one_line in f.readlines()]
    return lines

### conv1+block_1 block_2+block_3 
### block_4+block_5 block_6+block_7
layers_name = ["conv_1",            
               "block_1_conv_1",     
               "block_1_deconv_1",
               "block_1_conv_2",
               "block_2_conv_1",
               "block_2_deconv_1",
               "block_2_conv_2",
               "block_3_conv_1",
               "block_3_deconv_1",
               "block_3_conv_2",
               "block_4_conv_1",
               "block_4_deconv_1",
               "block_4_conv_2",
               "block_5_conv_1",
               "block_5_deconv_1",
               "block_5_conv_2",
               "block_6_conv_1",
               "block_6_deconv_1",
               "block_6_conv_2",
               "block_7_conv_1",
               "block_7_deconv_1",
               "block_7_conv_2",
               "block_8_conv_1",]

In [3]:
model = mobilenet(channels=[12,12,12,12,24,24,24,24,36,36,36,36,48,48,48,128])
model.load_weights("./checkpoints/tmp/cp-0005.hdf5")
print(model.summary())

batch_size = 256
num_epochs = 5
initial_learning_rate = 0.001

checkpoint_path = "./checkpoints/tmp_vgg_pruning/cp-{epoch:04d}.hdf5"
LOG_DIR = "./checkpoints/tmp_vgg_pruning_fitlogs/"

train_image_arr = np.load("./val_list_image_array.npy")
train_label_arr = np.load("./val_list_one_hot.npy")

val_image_arr = np.load("./val_list_image_array.npy")
val_label_arr = np.load("./val_list_one_hot.npy")

train_dataset = tf.data.Dataset.from_tensor_slices((train_image_arr, train_label_arr))
val_dataset   = tf.data.Dataset.from_tensor_slices((val_image_arr, val_label_arr))

train_dataset = train_dataset.shuffle(buffer_size=20000)
# train_dataset = train_dataset.repeat(2)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

val_dataset   = val_dataset.batch(batch_size)
val_dataset   = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

2022-01-18 14:25:06.284565: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-01-18 14:25:06.416145: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:84:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.635GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-01-18 14:25:06.416197: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-01-18 14:25:06.419091: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-01-18 14:25:06.421590: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2022-01-18 14:25:06.422021: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 50, 50, 12)   120         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 50, 50, 12)   48          conv_1[0][0]                     
__________________________________________________________________________________________________
block_1_conv_1 (Conv2D)         (None, 50, 50, 12)   156         batch_normalization[0][0]        
_______________________________________________________________________________________

In [4]:
all_mask = []    
for i in range(len(layers_name)):
    print('------',layers_name[i],'------')
    layer_conv = model.get_layer(name=layers_name[i])
    apoz_layer_conv = identify.get_apoz(model, layer_conv, train_image_arr)
    high_apoz_channels_conv = identify.high_apoz(apoz_layer_conv, "both")
    print("---mask---", high_apoz_channels_conv,"-------")
    all_mask.append(high_apoz_channels_conv)

------ conv_1 ------


2022-01-18 14:25:13.854614: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2022-01-18 14:25:15.384681: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


---mask--- [3 5] -------
------ block_1_conv_1 ------
---mask--- [0 3] -------
------ block_1_deconv_1 ------
---mask--- [ 2  7 10 11] -------
------ block_1_conv_2 ------
---mask--- [4 9] -------
------ block_2_conv_1 ------
---mask--- [ 4 10] -------
------ block_2_deconv_1 ------
---mask--- [0 2] -------
------ block_2_conv_2 ------
---mask--- [ 3 20 22] -------
------ block_3_conv_1 ------
---mask--- [ 4  6 12 13 14 18 19 22] -------
------ block_3_deconv_1 ------
---mask--- [ 0  7 13 18] -------
------ block_3_conv_2 ------
---mask--- [ 7 13 14 17 21] -------
------ block_4_conv_1 ------
---mask--- [ 3  6  7 16 19 20 23] -------
------ block_4_deconv_1 ------
---mask--- [ 1  2  7 12 22] -------
------ block_4_conv_2 ------
---mask--- [ 1  9 24 25 26 28 32 33] -------
------ block_5_conv_1 ------
---mask--- [ 2  3  4  5  8  9 10 12 13 16 19 20 24 25 27 28 30 33] -------
------ block_5_deconv_1 ------
---mask--- [ 9 13 14 19 21 23 28 33] -------
------ block_5_conv_2 ------
---mask-

### 手动选择需要删除的通道
* conv_1+block_1,而block_1里面只看dw的list，1 * 1的卷积不看

In [18]:
conv1_block_1   = [2,7,10,11]
block_2_block_3 = [0,7,13,18]
block_4_block_5 = [9,13,14,19,21,23,28,33]
block_6_block_7 = [4,9,10,17,24,25,26,28,32,34,37,40,42,44,45,47]
block_8_conv_1 = [0,2,3,5,6,8,10,17,18,19,20,23,24,25,28,32,33,39,40,41,43,48,50,55,60,61,62,64,66,67,68,72,75,79,80,81,
                  82,84,86,87,88,89,95,99,100,102,106,111,112,114]

In [19]:
conv1_block_1_mask = [i for i in range(12) if i not in conv1_block_1]
block_2_block_3_mask = [i for i in range(24) if i not in block_2_block_3]
block_4_block_5_mask = [i for i in range(36) if i not in block_4_block_5]
block_6_block_7_mask = [i for i in range(48) if i not in block_6_block_7]
block_8_conv_1_mask = [i for i in range(128) if i not in block_8_conv_1]

In [24]:
origin_weight = model.get_weights()
origin_layers = model.layers
new_weights = origin_weight

In [25]:
for i in range(len(origin_weight)):
    print(origin_weight[i].shape)

(3, 3, 1, 12)
(12,)
(12,)
(12,)
(12,)
(12,)
(1, 1, 12, 12)
(12,)
(12,)
(12,)
(12,)
(12,)
(3, 3, 12, 1)
(12,)
(12,)
(12,)
(12,)
(12,)
(1, 1, 12, 12)
(12,)
(12,)
(12,)
(12,)
(12,)
(1, 1, 12, 12)
(12,)
(12,)
(12,)
(12,)
(12,)
(3, 3, 12, 1)
(12,)
(12,)
(12,)
(12,)
(12,)
(1, 1, 12, 24)
(24,)
(24,)
(24,)
(24,)
(24,)
(1, 1, 24, 24)
(24,)
(24,)
(24,)
(24,)
(24,)
(3, 3, 24, 1)
(24,)
(24,)
(24,)
(24,)
(24,)
(1, 1, 24, 24)
(24,)
(1, 1, 24, 24)
(24,)
(24,)
(24,)
(24,)
(24,)
(3, 3, 24, 1)
(24,)
(24,)
(24,)
(24,)
(24,)
(1, 1, 24, 36)
(36,)
(36,)
(36,)
(36,)
(36,)
(1, 1, 36, 36)
(36,)
(36,)
(36,)
(36,)
(36,)
(3, 3, 36, 1)
(36,)
(36,)
(36,)
(36,)
(36,)
(1, 1, 36, 36)
(36,)
(36,)
(36,)
(36,)
(36,)
(1, 1, 36, 36)
(36,)
(36,)
(36,)
(36,)
(36,)
(3, 3, 36, 1)
(36,)
(36,)
(36,)
(36,)
(36,)
(1, 1, 36, 48)
(48,)
(48,)
(48,)
(48,)
(48,)
(1, 1, 48, 48)
(48,)
(48,)
(48,)
(48,)
(48,)
(3, 3, 48, 1)
(48,)
(1, 1, 48, 48)
(48,)
(48,)
(48,)
(48,)
(48,)
(1, 1, 48, 128)
(128,)
(1, 1, 128, 4)
(4,)


In [26]:
def change(current_shape, j, new_weights, origin_weight, i, conv1_2_mask, num=12):
    if (current_shape[j]==num and j==0) or (current_shape[j]==num and j==1):
        new_weights[i] = origin_weight[i][conv1_2_mask]
    if current_shape[j]==num and j==2:
        new_weights[i] = origin_weight[i][:,:,conv1_2_mask,:]
    if current_shape[j]==num and j==3:
        new_weights[i] = origin_weight[i][:,:,:,conv1_2_mask]
    
for i in range(len(origin_weight)):
    current_shape = new_weights[i].shape
    for j in range(len(current_shape)):
        change(current_shape, j, new_weights, origin_weight, i, conv1_block_1_mask, 12)
        change(current_shape, j, new_weights, origin_weight, i, block_2_block_3_mask, 24)
        change(current_shape, j, new_weights, origin_weight, i, block_4_block_5_mask, 36)
        change(current_shape, j, new_weights, origin_weight, i, block_6_block_7_mask, 48)
        change(current_shape, j, new_weights, origin_weight, i, block_8_conv_1_mask, 128)

In [27]:
for i in range(len(origin_weight)):
    print(origin_weight[i].shape)

(3, 3, 1, 8)
(8,)
(8,)
(8,)
(8,)
(8,)
(1, 1, 8, 8)
(8,)
(8,)
(8,)
(8,)
(8,)
(3, 3, 8, 1)
(8,)
(8,)
(8,)
(8,)
(8,)
(1, 1, 8, 8)
(8,)
(8,)
(8,)
(8,)
(8,)
(1, 1, 8, 8)
(8,)
(8,)
(8,)
(8,)
(8,)
(3, 3, 8, 1)
(8,)
(8,)
(8,)
(8,)
(8,)
(1, 1, 8, 20)
(20,)
(20,)
(20,)
(20,)
(20,)
(1, 1, 20, 20)
(20,)
(20,)
(20,)
(20,)
(20,)
(3, 3, 20, 1)
(20,)
(20,)
(20,)
(20,)
(20,)
(1, 1, 20, 20)
(20,)
(1, 1, 20, 20)
(20,)
(20,)
(20,)
(20,)
(20,)
(3, 3, 20, 1)
(20,)
(20,)
(20,)
(20,)
(20,)
(1, 1, 20, 28)
(28,)
(28,)
(28,)
(28,)
(28,)
(1, 1, 28, 28)
(28,)
(28,)
(28,)
(28,)
(28,)
(3, 3, 28, 1)
(28,)
(28,)
(28,)
(28,)
(28,)
(1, 1, 28, 28)
(28,)
(28,)
(28,)
(28,)
(28,)
(1, 1, 28, 28)
(28,)
(28,)
(28,)
(28,)
(28,)
(3, 3, 28, 1)
(28,)
(28,)
(28,)
(28,)
(28,)
(1, 1, 28, 32)
(32,)
(32,)
(32,)
(32,)
(32,)
(1, 1, 32, 32)
(32,)
(32,)
(32,)
(32,)
(32,)
(3, 3, 32, 1)
(32,)
(1, 1, 32, 32)
(32,)
(32,)
(32,)
(32,)
(32,)
(1, 1, 32, 78)
(78,)
(1, 1, 78, 4)
(4,)


In [28]:
new_model = model = mobilenet(channels=[8,8,8,8,20,20,20,20,28,28,28,28,32,32,32,78])
new_model.summary()

Model: "functional_49"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 50, 50, 8)    80          input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 50, 50, 8)    32          conv_1[0][0]                     
__________________________________________________________________________________________________
block_1_conv_1 (Conv2D)         (None, 50, 50, 8)    72          batch_normalization_22[0][0]     
______________________________________________________________________________________

In [29]:
new_model.set_weights(new_weights)

In [30]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate,
                                                            decay_steps=15,
                                                            decay_rate=0.95,
                                                            staircase=True
                                                            )

new_model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=initial_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']
              )

checkpointer = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                  save_weights_only=True,
                                                  verbose=1,
                                                  )

reduce_lr = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)

HISTORY = new_model.fit(train_dataset, 
                    epochs=num_epochs,
                    validation_data=val_dataset,
                    callbacks=[checkpointer, reduce_lr, tensorboard_callback],
                    shuffle=True,
                    )

2022-01-18 14:41:30.877229: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-01-18 14:41:30.877298: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1391] Profiler found 1 GPUs
2022-01-18 14:41:30.878127: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcupti.so.10.1'; dlerror: libcupti.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/fu04/xueluoyang/anaconda3/envs/py37/lib/python3.7/site-packages/cv2/../../lib64:/mnt/fu04/xueluoyang/cudnn:/usr/local/cuda-10.1/lib64:
2022-01-18 14:41:30.878267: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcupti.so'; dlerror: libcupti.so: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/fu04/xueluoyang/anaconda3/envs/py37/lib/python3.7/site-packages/cv2/../../lib64:/mnt/fu04/xueluoyang/cudnn:/usr/local/cuda-10.1/lib64:
2022-01-18 14:41

Epoch 1/5
 1/41 [..............................] - ETA: 0s - loss: 0.7824 - accuracy: 0.9219WARNING:tensorflow:From /mnt/fu04/xueluoyang/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


2022-01-18 14:41:35.808089: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-01-18 14:41:35.808206: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1441] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


 2/41 [>.............................] - ETA: 2s - loss: 0.7082 - accuracy: 0.9199WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0216s vs `on_train_batch_end` time: 0.1193s). Check your callbacks.


2022-01-18 14:41:35.893914: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-01-18 14:41:35.900630: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: ./checkpoints/tmp_vgg_pruning_fitlogs/train/plugins/profile/2022_01_18_14_41_35
2022-01-18 14:41:35.902799: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to ./checkpoints/tmp_vgg_pruning_fitlogs/train/plugins/profile/2022_01_18_14_41_35/fu04-SYS-4028GR-TR.trace.json.gz
2022-01-18 14:41:35.936168: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: ./checkpoints/tmp_vgg_pruning_fitlogs/train/plugins/profile/2022_01_18_14_41_35
2022-01-18 14:41:35.944579: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for memory_profile.json.gz to ./checkpoints/tmp_vgg_pruning_fitlogs/train/plugins/profile/2022_01_18_14_41_

41/41 [==============================] - ETA: 0s - loss: 0.6172 - accuracy: 0.9125
Epoch 00001: saving model to ./checkpoints/tmp_vgg_pruning/cp-0001.hdf5
41/41 [==============================] - 5s 124ms/step - loss: 0.6172 - accuracy: 0.9125 - val_loss: 0.6158 - val_accuracy: 0.9125
Epoch 2/5
41/41 [==============================] - ETA: 0s - loss: 0.5926 - accuracy: 0.9125
Epoch 00002: saving model to ./checkpoints/tmp_vgg_pruning/cp-0002.hdf5
41/41 [==============================] - 4s 103ms/step - loss: 0.5926 - accuracy: 0.9125 - val_loss: 0.6105 - val_accuracy: 0.9125
Epoch 3/5
41/41 [==============================] - ETA: 0s - loss: 0.5858 - accuracy: 0.9125
Epoch 00003: saving model to ./checkpoints/tmp_vgg_pruning/cp-0003.hdf5
41/41 [==============================] - 4s 102ms/step - loss: 0.5858 - accuracy: 0.9125 - val_loss: 0.6125 - val_accuracy: 0.9125
Epoch 4/5
41/41 [==============================] - ETA: 0s - loss: 0.5806 - accuracy: 0.9125
Epoch 00004: saving model to 